# 전통주 Recommendation system

In [3]:
import numpy as np
import pandas as pd
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval  # 문자열 파싱 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

## 1. 분석 동기 및 이유

<code>전통주란?</code>   
- 제조장 소재지 관할 지역이나 인접 지역에서 생산한 농산물을 주원료로 제조한 술은 지역특산주로 분류
- 주류부문 무형문화재 보유자나 식품명인이 면허를 받아 제조한 술(민속주) 등이 전통주
- 현행법상 주류 온라인으로 판매 불가. but 전통주 판매 활성화를 위해 2017년 7월부터 일반 온라인 쇼핑몰에서도 판매를 허용.

![nn](./주류현황.png)

주류 시장이 축소되는 와중에도 전통주 시장은 꾸준히 성장세를 보이고 있음👍   
주류시장 전체 차지하는 비율은 적어서 대중화는 아직이지만, 프리미엄 막걸리와 증류식 소주를 중심으로 관심도가 높아진 상황

➡ 제품 기본정보와 함께 온라인구매 후 남긴 리뷰, 음식정보 등을 활용하면 일반 소비자의 인식 수준에 맞춰서 다양한 추천 조건을 마련할 수 있을 것이라 생각

   ***   

## 2. Data 수집 과정

### 크롤링을 통한 전통주 관련 정보 수집

__< 네이버 지식백과 전통주 백과 >__

- https://terms.naver.com/list.naver?cid=58636&categoryId=58636
- 활용 데이터 : 제품 정보(ocr을 활용해 전통주 평가지표 수집)
- 사이트 소개 : 농림축산식품부, 한국농수산식품유통공사(aT)에서 제작지원을 받고 (사)한국전통민속주협회에서 컨텐츠를 편집 및 제공하여, 공신력이 있을 것으로 판단

__< 술담화 >__
- https://www.sooldamhwa.com
- 활용 데이터 : 제품 정보, 리뷰와 평점
- 사이트 소개 : 국내 최초 전통주 술 정기 구독 서비스 쇼핑몰로 전통주에 대한 다양한 콘텐츠 제공, 전통주 매니아들의 애정어린 리뷰 다수

__< 더 술 >__
-  https://thesool.com
- 활용 데이터 : 제품 정보
- 사이트 소개 : 농림축산식품부와 한국농수산식품유통공사가 전통주의 가치를 알리기 위해 운영하는 우리술 종합 정보 사이트

__< 우리술상회 >__
- https://smartstore.naver.com/woorisulgallery
- 활용 데이터 : 제품 리뷰와 평점
- 사이트 소개 : 전통주 전문 네이버 스마트 스토어

***

## 3. 데이터 전처리

### 1) OCR을 활용한 정량적 전통주 평가지표 수집

![ocr](./Naver_Liquor_Wiki/img_save/_감사그래프이미지datafontimagefal.png)

<code>OCR(광학문자인식)</code>   


__Vision AI__ : 이미지에서 유용한 정보를 도출하는 Auto ML Vision   
전통주제품의 맛 지표가 적힌 이미지파일에서 텍스트를 추출하기위해 google vision API를 가져와서 text detection을 활용 


➡ 추출한 텍스트로 맛 지표 정리 후 점수 직접 입력   

![before](./ocr전처리전.png)

In [4]:
df_wiki = pd.read_csv("./Naver_Liquor_Wiki/naver_wiki_total.csv")
df_wiki.drop(columns = 'Unnamed: 0', axis = 1, inplace = True)
df_wiki.head(5)

,제품명,향기,단맛,신맛,청량감,바디감,목넘김,균형감
0,1000억막걸리프리바이오,NaN,3.0,4.0,3.0,3.0,NaN,NaN
1,1000억유산균막걸리,NaN,3.0,5.0,5.0,3.0,NaN,NaN
2,1932새싹땅콩햅쌀막걸리,NaN,2.0,2.0,3.0,5.0,NaN,NaN
3,1932새싹땅콩흑미막걸리,NaN,3.0,1.0,1.0,4.0,NaN,NaN
4,33JU,4.0,3.0,NaN,NaN,1.0,1.0,NaN


### 2) 제품리스트 통일

➡ 사이트 별로 취급하는 제품이 다양하여, 제품명을 통일시키고 index를 매칭하는 방식으로 추천에 사용할 전통주리스트 통합

In [5]:
df_ver4 = pd.read_csv("./Naver_Liquor_Wiki/naver_wiki_product_ver4.csv")
df_ver4.drop(columns = ['Unnamed: 0', '향기', '단맛', '신맛', '청량감', '바디감', '목넘김',
       '담백', '탄닌감', '탄산', '주종', '도수%', '용량', '가격', '원재료', '원재료키워드', '제조사',
       '키워드(thesool)', '어울리는음식(thesool)'], axis = 1)

,index,제품명
0,0,1000억막걸리프리바이오
1,1,1000억유산균막걸리
2,2,1932새싹땅콩햅쌀막걸리
3,3,1932새싹땅콩흑미막걸리
4,4,33JU
...,...,...
528,528,회곡생동동주
529,529,회곡생막걸리
530,530,흑돈주
531,531,희양산막걸리15


###  3) column 선별

➡ 다양한 피처 중 null값이 적은 '단맛', '신맛', '청량감', '바디감', '도수'를 정량 평가지표로 선정   
나머지 특징 키워드를 정성 평가지표로 선정하였으며 정량적 피처 중 null값이 많은 경우 해당 특징 키워드로 추가

In [6]:
df_ver4.head(5)

,Unnamed: 0,index,제품명,향기,단맛,신맛,청량감,바디감,목넘김,담백,...,탄산,주종,도수%,용량,가격,원재료,원재료키워드,제조사,키워드(thesool),어울리는음식(thesool)
0,0,0,1000억막걸리프리바이오,NaN,3.0,4.0,3.0,3.0,NaN,NaN,...,NaN,탁주,5,750ml,3500,"쌀,밀,누룩,정제수","쌀,밀,",(주)국순당,NaN,NaN
1,1,1,1000억유산균막걸리,NaN,3.0,5.0,5.0,3.0,NaN,NaN,...,NaN,탁주,5,750ml,3200,"쌀,밀,누룩,정제수","쌀,밀,",(주)국순당,저도수,"적당한 감칠맛과 청량미가 있어 전, 회무침과 잘 어울린다."
2,2,2,1932새싹땅콩햅쌀막걸리,NaN,2.0,2.0,3.0,5.0,NaN,NaN,...,NaN,탁주,6,900ml,NaN,"쌀,땅콩농축액,입국,효모,고과당,정제효소제,정제수","쌀,땅콩,",(주)1932포천일동막걸리,NaN,NaN
3,3,3,1932새싹땅콩흑미막걸리,NaN,3.0,1.0,1.0,4.0,NaN,NaN,...,NaN,탁주,6,900ml,NaN,"쌀,흑미,땅콩농축액,입국,효모,고과당,정제효소제,정제수","쌀,땅콩,",(주)1932포천일동막걸리,NaN,마늘새우
4,4,4,33JU,4.0,3.0,NaN,NaN,1.0,1.0,NaN,...,NaN,증류주,18,500ml/200ml,NaN,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀","꿀,삼,",농업회사법인33가주식회사,NaN,NaN


In [7]:
df_ver4.columns

Index(['Unnamed: 0', 'index', '제품명', '향기', '단맛', '신맛', '청량감', '바디감', '목넘김',
       '담백', '탄닌감', '탄산', '주종', '도수%', '용량', '가격', '원재료', '원재료키워드', '제조사',
       '키워드(thesool)', '어울리는음식(thesool)'],
      dtype='object')

### 4) 원재료 키워드 선별

➡ 1차 전처리 후 전체 제품의 원재료 확인하여 2개 이상 반복되는 단어들 중 특징적인 재료를 키워드로 총 30개 선별하였음.

In [26]:
from collections import Counter
ingredients = []
df_ver4['원재료'].apply(lambda x : ingredients.extend(re.split(',', x)))
result = Counter(ingredients)
print(result)

Counter({'정제수': 300, '누룩': 185, '쌀': 169, '효모': 134, '찹쌀': 52, '국내산쌀': 46, '백미': 41, '아스파탐': 37, '국': 35, '주정': 33, '포도': 32, '젖산': 27, '멥쌀': 26, '구연산': 26, '입국': 25, '정제효소': 22, '소맥분': 20, '오미자': 19, '정제효소제': 16, '국내산찹쌀': 16, '과당': 15, '복분자': 15, '물엿': 14, '전통누룩': 14, '종국': 13, '효소': 12, '효모등': 12, '고과당': 11, '조제종국': 11, '보리': 10, '사과': 10, '구기자': 9, '국내산산머루': 9, '산양산삼': 8, '꿀': 8, '설탕': 8, '전분당': 8, '밀': 7, '벌꿀': 7, '올리고당': 7, '무수아황산': 7, '감': 6, '백설탕': 6, '곡자': 6, '팽화미': 6, '밀함유': 5, '인삼': 5, '생강': 5, '감초': 5, '밀가루': 5, '양조용포도': 5, '아스파탐등': 5, '정제주정': 5, '증류원액': 4, '재리종국화꽃': 4, '진달래': 4, '재래종솔잎': 4, '정제주': 4, '고구마': 4, '정제수등': 4, '밀누룩': 4, '효소제': 4, '과당등': 4, '죽력': 4, '거봉포도원액': 4, '우리밀누룩': 4, '물엿등': 4, '': 4, '젖산등': 4, '산수유': 4, '증류주': 4, '사과농축액': 4, '국내산포도': 4, '햅쌀': 4, '조효소제': 4, '물': 4, '쌀가루': 4, '홍삼농축액': 4, '국내산찹살': 4, '향온곡': 4, '아황산염': 4, '소르빈산염': 4, '효소처리스테비아': 3, '조효소재': 3, '정제효소등': 3, '오가피': 3, '멥살': 3, '맥아': 3, '호프펠렛': 3, '복분자과실': 3, '아세설팜칼륨': 3, '메타중아황산칼륨': 3, '옥수수전분': 3, 

In [8]:
dfi = pd.read_csv("./Naver_Liquor_Wiki/ingredients_keywords.csv")
in_list = dfi['0'].to_list()
in_list

['쌀',
 '찹쌀',
 '포도',
 '오미자',
 '복분자',
 '머루',
 '사과',
 '보리',
 '구기자',
 '꿀',
 '삼',
 '밀',
 '오디',
 '생강',
 '감초',
 '꽃',
 '솔잎',
 '고구마',
 '귤',
 '밤',
 '산수유',
 '땅콩',
 '오가피',
 '다래',
 '계피',
 '버섯',
 '감자',
 '복숭아',
 '약재',
 '베리']

### 5) 결측치 처리

➡ __KNN Imputer__ 를 사용해 결측치를 유사한 컬럼과 비슷한 값으로 대체

In [10]:
df = pd.read_csv("./Traditional_liquor_df_final.csv")
df1 = df[['단맛', '신맛', '청량감', '바디감','도수%']]
df1[3:9]

,단맛,신맛,청량감,바디감,도수%
3,3.0,1.0,1.0,4,6.0
4,3.0,NaN,NaN,1,18.0
5,3.0,NaN,NaN,2,17.5
6,2.0,NaN,NaN,3,25.0
7,2.0,2.0,2.0,3,12.0
8,4.0,2.0,3.0,2,16.0


In [11]:
imputer = KNNImputer(n_neighbors=5)
dfc = pd.DataFrame(imputer.fit_transform(df1), columns=['단맛','신맛','청량감','바디감','도수%'])
dfc[3:9]

,단맛,신맛,청량감,바디감,도수%
3,3.0,1.0,1.0,4.0,6.0
4,3.0,2.2,2.0,1.0,18.0
5,3.0,2.4,2.2,2.0,17.5
6,2.0,0.6,1.8,3.0,25.0
7,2.0,2.0,2.0,3.0,12.0
8,4.0,2.0,3.0,2.0,16.0


### 6) 키워드 정리

➡ 단맛, 신맛, 청량감, 바디감, 도수를 제외한 정량지표와 정성지표를 keyword화 하여 한 컬럼에 입력

In [12]:
dfk = pd.read_csv("./cbf_database3.csv")
dfk

,index,제품명,keyword
0,0,1000억막걸리프리바이오,탁주 쌀 밀 저도수
1,1,1000억유산균막걸리,탁주 쌀 밀 저도수
2,2,1932새싹땅콩햅쌀막걸리,탁주 쌀 땅콩 저도수
3,3,1932새싹땅콩흑미막걸리,탁주 쌀 땅콩 저도수
4,4,33JU,향기 증류주 꿀 삼
...,...,...,...
528,528,회곡생동동주,탁주 쌀 저도수
529,529,회곡생막걸리,탁주 쌀 저도수
530,530,흑돈주,목넘김 기타주류
531,531,희양산막걸리15,탁주 쌀 무감미료 저도수


### 7) 리뷰 키워드 정리

1. Konlpy의 OKT 를 사용해 단어 토큰화   
2. CountVectorizer로 절대 빈도수 추출   
3. TF/IDF 전체 문서 대비 특정 단어 빈도수 추출   
4. 비교 후 단어의 중요도 판별해 불용어 처리   
5. 단어들을 카테고리화 하여 키워드로 추가

In [27]:
rv = pd.read_csv("reviews.csv")
rv.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
rv[rv['total_index'] == 21]

,total_index,제품명,customerID,score,comments
114,21,감홍로백자,9414***,5.0,포장도 꼼꼼하고 배송도 빠르게 왔습니다! 만족해여
115,21,감홍로백자,hst8***,5.0,선물용으로 샀습니다. 포장은 잘 되어서 왔습니다. 감사히 잘 선물하겠습니다.
116,21,감홍로백자,si****,5.0,매우 만족스럽습니다
117,21,감홍로백자,rods***,4.0,명절 선물로 미리보내드렸어요 도자기에 술잔까지 좋은분위기를 낼수잇어서 너무좋은시간...
118,21,감홍로백자,wari*******,5.0,단언컨데 감홍로는 제가 먹어봤던 전통주 중에 개성이 제일 빼어난 술입니다. 선물 포...
...,...,...,...,...,...
517,21,감홍로백자,drog*****,5.0,맛잇어요!! 좋은기분입니다
518,21,감홍로백자,love*****,5.0,친구 아버님한테 선물로 드렸습니다 좋아하셨으면 좋겠습니다
519,21,감홍로백자,lemo*******,5.0,포장 꼼꼼하고 배송빨라요
520,21,감홍로백자,ento******,5.0,한달사용기선물로 샀어요 좋아요


![cv](countvec_image.png)

In [14]:
dfk1 = pd.read_csv("keyword_cleansed.csv")
dfk1.drop(columns = ['review'], axis = 1)

,total_index,review2
0,12,NaN
1,20,커플 반주 안주 선물 파티 가족
2,21,커플 친구 명절 부모님 안주 선물 기념일 추천 가족 행사
3,24,선물 명절 친구 추천
4,26,커플 디저트 친구 캠핑 명절 반주 안주 부모님 선물 추천 파티 가족 크리스마스 혼술 계절
...,...,...
185,471,NaN
186,473,선물 커플 디저트
187,493,커플 디저트 친구 안주 선물 추천
188,524,선물 추천 부모님


### 8) 키워드 간 병합 진행

In [15]:
dfk2 = pd.merge(dfk,dfk1,left_on='index',right_on='total_index',how='outer')
dfk2.fillna("", inplace= True)

dfk2['keywords'] = dfk2['keyword'] + " " + dfk2['review2']

dfk2.drop(columns = ['keyword','total_index','review','review2'], axis = 1, inplace = True)
dfk2[470 :475]

,index,제품명,keywords
470,470,티나Tina,향기 목넘김 기타주류 이색전통주
471,471,펀치쌀바나나,탄산 기타주류 쌀 저도수
472,472,편백숲산소막걸리,탁주 쌀 사과 저도수
473,473,편백숲산소막걸리딸기스파클링,탄산 탁주 쌀 찹쌀 이색전통주 선물 커플 디저트 저도수 선물 커플 디저트
474,474,편백숲산소막걸리순수령,탁주 쌀 찹쌀 홈술 저도수


### 9) 최종 데이터 베이스

In [16]:
df_final = pd.read_csv("df_final.csv")
df_final

,index,제품명,단맛,신맛,청량감,바디감,도수,용량,가격,keyword,제조사,원재료
0,0,1000억막걸리프리바이오,3.0,4.0,3.0,3.0,5.0,750ml,3500,탁주 쌀 밀 저도수,(주)국순당,"쌀,밀,누룩,정제수"
1,1,1000억유산균막걸리,3.0,5.0,5.0,3.0,5.0,750ml,3200,탁주 쌀 밀 저도수,(주)국순당,"쌀,밀,누룩,정제수"
2,2,1932새싹땅콩햅쌀막걸리,2.0,2.0,3.0,5.0,6.0,900ml,15000,탁주 쌀 땅콩 저도수,(주)1932포천일동막걸리,"쌀,땅콩농축액,입국,효모,고과당,정제효소제,정제수"
3,3,1932새싹땅콩흑미막걸리,3.0,1.0,1.0,4.0,6.0,900ml,15000,탁주 쌀 땅콩 저도수,(주)1932포천일동막걸리,"쌀,흑미,땅콩농축액,입국,효모,고과당,정제효소제,정제수"
4,4,33JU,3.0,2.2,2.0,1.0,18.0,200ml,23000,향기 증류주 꿀 삼,농업회사법인33가주식회사,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀"
...,...,...,...,...,...,...,...,...,...,...,...,...
528,528,회곡생동동주,3.0,2.0,3.0,3.0,7.0,"1,200ml",1300,탁주 쌀 저도수,회곡양조장,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨"
529,529,회곡생막걸리,2.0,2.0,2.0,4.0,6.0,750ml,1300,탁주 쌀 저도수,회곡양조장,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨"
530,530,흑돈주,2.4,2.6,1.4,2.0,16.0,365ml,12000,목넘김 기타주류,왕지케양조장(제주본초협동조합),"황칠나무,정제수,,효모"
531,531,희양산막걸리15,3.0,4.0,1.0,5.0,15.0,500ml,9500,탁주 쌀 무감미료 저도수,두술도가,"쌀,누룩,정제수"


***

## 4. Recommendation System

처음 이용하는 사용자는 주종,당도,도수 등 선호도를 입력   
기존 구매이력이 있는 사용자는 최근 구매했던 제품이나 선호하는 제품을 참고   

__전통주 중 하나를 선택하면 그와 유사한 전통주를 출력🙌__



정량적인 지표와 정성적인 지표를 동시에 포함하기 위해 단맛, 신맛, 청량감, 바디감의 정량적인 지표의 근삿값 상위 30개를 필터링 한 후 키워드 기반으로 재정렬

### 1) 정량적 특징 기반 CBF

➡ 코사인 유사도를 통해 정량적으로 단맛, 신맛, 청량감, 바디감, 도수가 비슷한 제품을 출력

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

df_for_cossim_T = dfc.transpose()

item_sim = cosine_similarity(dfc, dfc)
item_sim_df = pd.DataFrame(data=item_sim, index=df_for_cossim_T.columns, columns=df_for_cossim_T.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(533, 533)


,0,1,2,3,4,5,6,7,8,9,...,523,524,525,526,527,528,529,530,531,532
0,1.000000,0.980841,0.933700,0.886142,0.764967,0.792571,0.707600,0.840221,0.820341,0.986117,...,0.799071,0.714169,0.672147,0.835175,0.789058,0.949071,0.939826,0.789058,0.861337,0.914425
1,0.980841,1.000000,0.892329,0.796926,0.688839,0.716546,0.622257,0.766903,0.750265,0.953206,...,0.717209,0.633632,0.586604,0.746248,0.708516,0.904291,0.881409,0.708516,0.773973,0.852090
2,0.933700,0.892329,1.000000,0.955778,0.791300,0.825434,0.784955,0.890291,0.845878,0.960769,...,0.821468,0.776017,0.749723,0.876182,0.821489,0.962102,0.990742,0.821489,0.899647,0.930421


In [18]:
c0 = pd.DataFrame(item_sim_df[16].sort_values(ascending=False)[0:30])
c0['index'] = c0.index
c1 = pd.merge(c0, dfc, left_on = 'index', right_on= dfc.index, how = 'inner')
c1.head(5)

,16,index,단맛,신맛,청량감,바디감,도수%
0,1.000000,16,2.0,2.0,2.4,2.0,14.0
1,0.999694,43,2.0,2.0,2.0,2.0,13.0
2,0.999261,373,3.0,3.0,3.0,3.0,18.0
3,0.999261,508,2.0,2.0,2.0,2.0,12.0
4,0.999243,274,2.0,2.0,2.0,2.0,15.0


### 2) 정성적 keyword 기반 CBF

In [19]:
from ast import literal_eval  # 문자열 파싱 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))  
# min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2

dfk3 = pd.merge(c1, dfk2, on='index',how='inner')
key_mat = count_vect.fit_transform(dfk3['keywords'])

print(key_mat.shape)
# print(key_mat)

(30, 109)


In [20]:
dfk3.head(5)

,16,index,단맛,신맛,청량감,바디감,도수%,제품명,keywords
0,1.000000,16,2.0,2.0,2.4,2.0,14.0,감사,목넘김 담백 약주/청주 쌀 선물 선물 추천 저도수 선물 추천
1,0.999694,43,2.0,2.0,2.0,2.0,13.0,고창선운산땡큐블루베리주,과실주 베리 소용량 저도수
2,0.999261,373,3.0,3.0,3.0,3.0,18.0,이상헌약주,약주/청주 쌀 진한맛
3,0.999261,508,2.0,2.0,2.0,2.0,12.0,행운주,약주/청주 밀 저도수
4,0.999243,274,2.0,2.0,2.0,2.0,15.0,순향주,약주/청주 쌀 파티 저도수


In [32]:
count_vect = CountVectorizer(min_df=1, ngram_range=(1, 3))  # min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
key_mat = count_vect.fit_transform(dfk3['keywords'])
print(key_mat.shape)
from sklearn.metrics.pairwise import cosine_similarity
key_sim = cosine_similarity(key_mat, key_mat)
print(key_sim.shape)

(30, 193)
(30, 30)


In [22]:
# 대칭 행렬
key_sim_sorted_ind = key_sim.argsort()[:, ::-1] # ::-1 : 역순으로 정렬
print(key_sim_sorted_ind[:5])
title_sool = dfk3[dfk3['제품명'] == '감사']

sool_index = title_sool.index.values

similar_indexes = key_sim_sorted_ind[sool_index, :10]
similar_indexes
# 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
# dataframe에서 index로 사용하기 위해서 2차원 array를 1차원 array로 변경
similar_indexes = similar_indexes.reshape(-1)
print(similar_indexes)

[[ 0 24 28 17 10 26  3 11 20 19 13  4  2  8 14 18  7 25  6 22 23 16 21 27
   1 15 12  9  5 29]
 [ 1 16 12 26  3 11 15  6 27  4 17 14  0 28 20 24 18 22 25  2 23  5  8  7
  19  9 10 21 13 29]
 [ 2 11 26  3 10  4 17  0 13 15 24  6  7  8  9  1  5 29 12 28 16 18 19 20
  21 22 23 25 27 14]
 [26  3 11  4  2 17 10  0 13 24 16 27  1  6 28 20 14 12 15 18  9  8  7 19
   5 21 22 23 25 29]
 [ 4 26  3 11  2 17 10  0  6 13 24 16 20 27  1 28 14 12 15 18  9  8  7 19
   5 21 22 23 25 29]]
[ 0 24 28 17 10 26  3 11 20 19]


In [23]:
k1 = dfk3.iloc[similar_indexes]
k1

,16,index,단맛,신맛,청량감,바디감,도수%,제품명,keywords
0,1.000000,16,2.0,2.0,2.4,2.0,14.0,감사,목넘김 담백 약주/청주 쌀 선물 선물 추천 저도수 선물 추천
24,0.996405,86,3.0,2.0,2.0,3.0,15.0,녹파주,담백 약주/청주 쌀 찹쌀 선물 커플 명절 선물 추천 가족 저도수 커플 명절 선물 추...
28,0.995554,146,3.0,2.6,1.8,3.0,15.0,매실원주15,기타주류 꿀 캠핑 명절 부모님 선물 추천 저도수 캠핑 명절 부모님 선물 추천
17,0.996886,189,2.0,3.0,3.0,3.0,15.0,방문주약주,약주/청주 쌀 찹쌀 선물 저도수
10,0.998032,241,3.0,2.0,3.0,3.0,16.0,서주16,약주/청주 감자 선물
26,0.996021,268,2.0,1.0,2.0,2.0,10.0,소백산청동동주,약주/청주 쌀 저도수
3,0.999261,508,2.0,2.0,2.0,2.0,12.0,행운주,약주/청주 밀 저도수
11,0.997758,144,3.0,2.0,2.0,2.0,15.0,맑은술해밀,약주/청주 쌀 저도수
20,0.996494,502,1.0,1.0,2.0,2.0,12.0,해창12도생막걸리,탁주 쌀 찹쌀 복날 부모님 선물 파티 추천 저도수 복날 부모님 선물 파티 추천
19,0.996572,82,2.0,2.4,1.6,3.0,16.0,녹고의눈물,기타주류 오가피 커플 친구 명절 반주 안주 부모님 선물 추천 커플 친구 명절 반주 ...


### 최종 함수로 정리

In [24]:
def traditional_name(aaa) :
    a = dfk.loc[dfk['제품명']==aaa,'index']
    a = int(a)
    
    df_for_cossim_T = dfc.transpose()

    item_sim = cosine_similarity(dfc, dfc)
    item_sim_df = pd.DataFrame(data=item_sim, index=df_for_cossim_T.columns, columns=df_for_cossim_T.columns)

    c0 = pd.DataFrame(item_sim_df[a].sort_values(ascending=False)[0:30])
    c0['index'] = c0.index
    c1 = pd.merge(c0, dfc, left_on = 'index', right_on= dfc.index, how = 'inner')

    count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))  
    # min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2

    dfk3 = pd.merge(c1, dfk2, on='index',how='inner')
    key_mat = count_vect.fit_transform(dfk3['keywords'])


    count_vect = CountVectorizer(min_df=1, ngram_range=(1, 3))  # min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
    key_mat = count_vect.fit_transform(dfk3['keywords'])
    key_sim = cosine_similarity(key_mat, key_mat)

    # 대칭 행렬
    key_sim_sorted_ind = key_sim.argsort()[:, ::-1] # ::-1 : 역순으로 정렬

    title_sool = dfk3[dfk3['index'] == a]

    sool_index = title_sool.index.values

    similar_indexes = key_sim_sorted_ind[sool_index, :10]
    similar_indexes
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
    # dataframe에서 index로 사용하기 위해서 2차원 array를 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)
   
    k1 = dfk3.iloc[similar_indexes]
    return k1[0:11]


In [33]:
a = input("전통주 이름 : ")
traditional_name(a)

,530,index,단맛,신맛,청량감,바디감,도수%,제품명,keywords
1,1.000000,527,2.4,2.6,1.4,2.0,16.0,황칠주,목넘김 기타주류
2,1.000000,530,2.4,2.6,1.4,2.0,16.0,흑돈주,목넘김 기타주류
0,1.000000,319,2.4,2.6,1.4,2.0,16.0,어우야,목넘김 기타주류 홈술
20,0.996795,512,2.0,2.0,1.8,2.0,20.0,호담,기타주류 삼 삼(蔘)류
4,0.999156,114,3.0,2.6,2.0,2.0,20.0,동강더덕주,향기 기타주류 구기자 감초 선물 부모님 선물 부모님
26,0.996430,16,2.0,2.0,2.4,2.0,14.0,감사,목넘김 담백 약주/청주 쌀 선물 선물 추천 저도수 선물 추천
11,0.997766,82,2.0,2.4,1.6,3.0,16.0,녹고의눈물,기타주류 오가피 커플 친구 명절 반주 안주 부모님 선물 추천 커플 친구 명절 반주 ...
13,0.997498,144,3.0,2.0,2.0,2.0,15.0,맑은술해밀,약주/청주 쌀 저도수
3,0.999219,523,3.2,3.0,1.4,2.0,17.0,황금보리소주17,증류주 보리 선물 친구 선물 친구
5,0.999133,80,3.0,3.0,2.0,3.0,18.5,남산애포트와인,과실주 포도


***

## 5. 결론

➡ 개인화된 전통주 추천을 통해 전통주 시장 활성화를 기대

### 아쉬운 점

- 제품구매 개수가 많은 유저가 소수여서 CF 기반 개인화 추천이 어려움 : 구매하지 않은 제품의 예측평점이 낮게 나옴
- 다양한 데이터를 크롤링 하였으나 사용하지 못한 피쳐들이 있어 아쉬움
- 초기 기획에서는 특정 음식에 어울리는 전통주를 추천하는 시스템을 구상하였으나 음식에 대한 데이터의 부족으로 다양한 추천 조건을 구현하지 못해 아쉬움
- 리뷰 키워드화 과정 중 Topic modeling 사용하였으나 성능이 저조

### 추후 보완점

- 리뷰 단어를 카테고리화 하여 키워드로 사용하는 과정에 단어 가중치 부여 및 자동화가 필요
- 초기 input을 구현하지 못함